# Ciencia De Datos Aplicada: Segundo Entregable

## Recopilación y Preparación de Datos

**Equipo:** Uriel Arias - 63504, Cristian Tepedino - 62830

**Nombre del proyecto**: Sistema de predicción de resultados de
carreras de Fórmula 1